In [ ]:
from requests import get as r_get
from json import loads
from csv import DictWriter
from schemas import Vacancies, Vacancy

In [ ]:
def get_vacancy(id: int) -> Vacancy:
    """_summary_

    Args:
        id (int): _description_

    Returns:
        Vacancy: _description_
    """
    with r_get(f"https://api.hh.ru/vacancies/{id}") as req:
        data = loads(req.content.decode())
        print(f"Вакансия добавлена: {data.get('name')}")
        v_data = Vacancy.parse_obj(data)
        return v_data

In [ ]:
def get_initial_data() -> Vacancies:
    params = {
        "text": "NAME:Backend",  # NAME:Backend Текст фильтра. В имени должно быть слово "Backend"
        "area": 113,  # Поиск ощуществляется по вакансиям России
        "page": 0,  # Индекс страницы поиска на HH
        "per_page": 100,  # Кол-во вакансий на 1 странице
    }
    with r_get("https://api.hh.ru/vacancies", params) as req:
        return Vacancies.parse_obj(loads(req.content.decode()))

In [ ]:
def getPage(page: int = 0) -> list[Vacancy]:
    """
    Создаем метод для получения страницы со списком вакансий.

    Args:
        page (int, optional): Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница

    Returns:
        str: вакансии
    """
    params = {
        "text": "NAME:Backend",  # NAME:Backend Текст фильтра. В имени должно быть слово "Backend"
        "area": 113,  # Поиск ощуществляется по вакансиям России
        "page": page,  # Индекс страницы поиска на HH
        "per_page": 100,  # Кол-во вакансий на 1 странице
    }
    with r_get("https://api.hh.ru/vacancies", params) as req:  # Посылаем запрос к API
        data = loads(req.content.decode()).get("items", [])
    return [get_vacancy(item.get("id")) for item in data]

In [ ]:
all_vacancies = []
first_page = get_initial_data()
pages = first_page.pages
for page in range(1, pages):
    all_vacancies += getPage(page)
    print(f"Вакансии: {page}/{pages} ({len(all_vacancies)})")

In [ ]:
fieldnames = list(all_vacancies[0].schema()["properties"].keys())

In [ ]:
with open("vacancies.csv", "w", newline="") as output_file:
    fieldnames = list(all_vacancies[0].schema()["properties"].keys())
    dict_writer = DictWriter(output_file, fieldnames)
    dict_writer.writeheader()
    for vacancy in all_vacancies:
        vacancy = vacancy.dict()
        if vacancy.get("salary"):
            from_ = vacancy.get("salary").get("start") or 0
            to_ = vacancy.get("salary").get("to") or 0
            print(from_, to_, vacancy.get("salary"))
            average_salary = (from_ + to_) // 2
            if from_ == 0:
                average_salary = to_
            if to_ == 0:
                average_salary = from_
            vacancy["salary"] = average_salary if vacancy.get("salary").get("currency") == "RUR" else average_salary * 80
        vacancy["experience"] = vacancy.get("experience").get("name")
        vacancy["area"] = vacancy.get("area").get("name")
        vacancy["key_skills"] = ", ".join([skill.get("name") for skill in vacancy["key_skills"]])
        dict_writer.writerow(vacancy)